In [2]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import pandas as pd
import requests
from time import strftime
import datetime
import time
from pandas import ExcelWriter
import re
import pyodbc
import asyncio
from selenium.webdriver.common.action_chains import ActionChains
import numpy as np
import pdb
#pdb.set_trace()
from ipywidgets import IntProgress
from IPython.display import display
import os
from selenium.webdriver.support import expected_conditions
from IPython.display import clear_output
from datetime import timedelta

In [3]:
TIEMPO_DE_ESPERA = 4

start_ = datetime.datetime.now()
fecha = datetime.date.today()
hoy = fecha.strftime('%Y/%m/%d')



In [4]:
#fecha='2019-12-18'

In [5]:
def createDriver():
    user_agent = 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.50 Safari/537.36'

    chrome_options = Options()
    chrome_options.add_argument("--headless")
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument(f'user-agent={user_agent}')
    chrome_options.add_argument("--window-size=1625x2444")
    # options.headless = True
    return webdriver.Chrome(options = chrome_options)

In [ ]:
#fecha = fecha - timedelta(days=2)

In [ ]:
dfLinks = pd.read_excel(f'dfLinks_{fecha}.xlsx')

In [10]:
dfLinks = pd.read_excel(f'dfLinks_2019-12-31.xlsx')

In [11]:
len(dfLinks[dfLinks['sexo']=='Kids'])

8

In [12]:
kids = dfLinks.loc[dfLinks['sexo'] == 'Kids'].reset_index()

In [ ]:
nocargaron = []
lista_productos = []

browser = createDriver()
for index, row in kids.iterrows():
    
    #requests de la url //ropa mujer//hombre//kids
    try:
        browser.get(row['url'])
    except:
        browser.quit()
        browser = createDriver()
        time.sleep(1)
        continue
    

    last_height = browser.execute_script("return document.body.scrollHeight")
    while True:

        browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        try:
            #boton para cargar más items
            browser.find_element_by_class_name('load-more-products').find_element_by_tag_name('button').click()
        except:
            pass
        time.sleep(TIEMPO_DE_ESPERA)
        new_height = browser.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

    time.sleep(.5)
    
    CANTIDAD_PRODUCTOS = len(browser.find_elements_by_class_name('product-item'))
    
    style = {'description_width': 'initial'}
    test = IntProgress(style=style, min = 0, max = CANTIDAD_PRODUCTOS, description=row['tipo']+','+row['sexo'], bar_style='danger',)    
    display(test)
    #CANTIDAD DE PRODUCTOS
    #_ no lo uso porque deja de estar en el DOM ya que hago click y se pierde el elemento, solo necesito el indice,
    for index_producto in range(CANTIDAD_PRODUCTOS):
        
        
        pos = index_producto+1
        try:
            
            browser.find_elements_by_class_name('hm-product-item')[0].find_element_by_tag_name('a').click()
        except:
            continue

        try:
            #WebDriverWait(browser, 100).until(EC.presence_of_element_located((By.CLASS_NAME,'group')))
            WebDriverWait(browser, 90).until(EC.presence_of_element_located((By.CLASS_NAME,'mini-slider')))
        except:
            browser.get(browser.current_url)
            time.sleep(5)

        try:

            cantidad_colores = len(browser.find_element_by_class_name('group').find_elements_by_class_name('list-item'))
        except:
            nocargaron.append([browser.current_url])
            browser.execute_script('history.back();')
            continue

        for color in range(cantidad_colores):


            #click en el color.

            elemento_auxiliar = browser.find_element_by_class_name('group').find_elements_by_class_name('list-item')[color].find_element_by_tag_name('a')
            elemento_auxiliar.click()

            nombre_color = elemento_auxiliar.get_attribute('title')
            id_producto = elemento_auxiliar.get_attribute('data-articlecode')
            descripcion = browser.find_element_by_css_selector('.primary.product-item-headline').text
            precio = browser.find_element_by_css_selector('.price.parbase').text
            url_producto = browser.current_url
            img = browser.find_element_by_class_name('product-detail-main-image-container').find_element_by_tag_name('img').get_attribute('src')

            lista_productos.append([pos,
                                    id_producto,
                                    descripcion,
                                    nombre_color,
                                    row['tipo'],
                                    precio,
                                    row['sexo'],
                                    url_producto,
                                    img,
                                    row['url']])

        browser.execute_script('history.back();')
        test.value +=1
        test.description = f"± {int(CANTIDAD_PRODUCTOS)*15}: {test.value} / {CANTIDAD_PRODUCTOS}"

    test.bar_style='success'
    clear_output() 

end_ = datetime.datetime.now()


IntProgress(value=0, bar_style='danger', description='Newborn 0-9m,Kids', max=761, style=ProgressStyle(descrip…

In [25]:
print(browser.current_url)

https://www2.hm.com/en_us/kids/products/newborn-size-0-9m.html?product-type=kids_newborn_all&sort=stock&image-size=small&image=stillLife&offset=0&page-size=792


In [30]:
browser.save_screenshot('asd.png')

True

In [22]:
browser.execute_script('history.back();')

In [17]:
browser.quit()

In [52]:
lista_productos[-2]

['0771826003',
 'Rib-knit Wrap Dress',
 'Red',
 'Dresses',
 '$59.99',
 'Mujer',
 'https://www2.hm.com/en_us/productpage.0771826003.html',
 '//lp2.hm.com/hmgoepprod?set=source[/4d/fe/4dfe78723f0547659d158a99c65a3fa859c88425.jpg],origin[dam],category[ladies_dresses_mididresses],type[LOOKBOOK],res[m],hmver[1]&call=url[file:/product/main]']

In [ ]:
browser.execute_script(f'''

$("#dropdown-colorWithNames label span:contains('{CLICK_COLOR}')").click()

''')

In [19]:
len(lista_productos)

11619

In [17]:
browser.quit()

In [22]:
df = pd.DataFrame(lista_productos)

In [23]:
df.rename(columns={0:'pos',
                  1:'sku',
                  2:'descripcion',
                  3:'color',
                  4:'tipo',
                  5:'precio',
                  6:'sexo',
                  7:'url',
                  8:'jpg',
                  9:'pagina_scraper'},inplace=True)

In [20]:
#df['precio'] = df['precio'].apply(lambda x: float(x.split()[1]))

In [31]:
for i in df['tipo'].unique():
    print(len(df[df['tipo'] == i]))
    

2354
1251
320
4565
1026
2103


In [32]:
df.to_excel(f'./Salida/HyM_Kids{str(fecha)}.xlsx')

In [33]:
%hist

from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import pandas as pd
import requests
from time import strftime
import datetime
import time
from pandas import ExcelWriter
import re
import pyodbc
import asyncio
from selenium.webdriver.common.action_chains import ActionChains
import numpy as np
import pdb
#pdb.set_trace()
from ipywidgets import IntProgress
from IPython.display import display
import os
from selenium.webdriver.support import expected_conditions
from IPython.display import clear_output
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from bs4 import B